# Named Entity Recognition

## Reading JSON string file into a Python array of dictionaries

In [1]:
import json

In [2]:
with open("nyt_articles2.json", 'r') as f:
    newsfeeds = json.load(f)

len(newsfeeds)

1158

In [3]:
import pandas as pd
df = pd.DataFrame(newsfeeds)

In [6]:
#choosing a random news article
text = df.loc[1, "Snippet"]
print(text)

Chameleons or beauties, star turns or character roles — these are the performers who have outshone all others on the big screen in the last 20 years.


## Step 2. Refinitiv Open Calais
Follow directions to set up one of the Information Extraction services below, and write a Python program implementing API calls to extract Company/Organization and Geo entities from  the article chosen in Step 1. Print all the recognized entities in (entity, label) tuple format. 

Reference: https://developers.refinitiv.com/en/article-catalog/article/intelligent-tagging-extract-information-api-response

https://medium.com/@kaljuvee/news-named-entity-extraction-ner-and-sentiment-analysis-4f02d9bbb9e3

### 2.1 Python program implementing API calls

In [15]:
# step 1: import libraries
import requests

step 2 if using URL:
fetch all URLs

In [69]:
df.keys()

Index(['Headline', 'Abstract', 'Lead_Paragraph', 'Snippet', 'Published_date',
       'Author', 'News_desk', 'URL', 'Source'],
      dtype='object')

In [75]:
all_URL = []
for index, row in df.iterrows():
    all_URL.append(row["URL"])

In [84]:
all_headlines = []
for index, row in df.iterrows():
    all_headlines.append(row["Headline"])

In [85]:
all_snippets = []
for index, row in df.iterrows():
    all_snippets.append(row["Snippet"])

In [92]:
# print(len(all_URL))
# print(len(all_headlines))
# print(len(all_snippets))

1158
1158
1158


Looping (step 2-4):

In [102]:
all_JsonResponses = [] 

for url in all_URL:
    url = url
    headers = {}
    HTMLResponse = requests.request('GET', url, headers=headers)
    contentText = HTMLResponse.text
    headType = 'text/html'
    
    token = 'nWR9RbP0mXi3TiZuTcrHeRGtalvCMp64'
    url = 'https://api-eit.refinitiv.com/permid/calais'
    payload = contentText.encode('utf8')
    headers = {
        'Content-Type': headType,
        'x-ag-access-token': token,
        #'x-calais-selectiveTags': 'Company,CompanyLocation, Movie, Organization, Person',
        'outputformat': 'application/json'
        }

    TRITResponse = requests.request('POST', url, data=payload, headers=headers)
    TRITTextResponse = TRITResponse.text
    
    TRITJsonResponse = json.loads(TRITTextResponse)

    all_JsonResponses.append(TRITJsonResponse)
        
with open("NER_Tagging.json", "w") as json_file:
    json.dump(all_JsonResponses, json_file, indent=4)    

In [20]:
url = df.loc[180, "URL"]

In [22]:
# Step 2: Provide input content in url. 
url = url
headers = {}
HTMLResponse = requests.request('GET', url, headers=headers)
contentText = HTMLResponse.text
headType = 'text/html'

In [60]:
# Step 3: Trigger the Intelligent Tagging API and get the tagging response.
token = 'nWR9RbP0mXi3TiZuTcrHeRGtalvCMp64'
url = 'https://api-eit.refinitiv.com/permid/calais'
payload = contentText.encode('utf8')
headers = {
    'Content-Type': headType,
    'x-ag-access-token': token,
    #'x-calais-selectiveTags': 'Company,CompanyLocation, Movie, Organization, Person',
    'outputformat': 'application/json'
    }
 
TRITResponse = requests.request('POST', url, data=payload, headers=headers)
TRITTextResponse = TRITResponse.text

# print(TRITTextResponse)

In [61]:
# Step 4: Load content into JSON object 
TRITJsonResponse = json.loads(TRITTextResponse)
textR = TRITJsonResponse["doc"]['info']['document']

# print(textR)

In [67]:
TRITJsonResponse

{'doc': {'info': {'id': 'http://id.opencalais.com/YEX*2OLtezMlsdictcdN5Q',
   'ontology': 'http://trit-us-east-1-sharedamd.int.refinitiv.com/owlschema/0.5.2/onecalais.owl.allmetadata.xml',
   'docId': 'http://d.opencalais.com/dochash-1/d8f174c9-c427-3a1d-8b2d-beb298aff341',
   'document': '<body> By MEKADO MURPHY\nAUG. 31, 2016\nIt may have been a season of superheroes, but moviegoers have been talking about these seven sleepers, indie features that didn\'t make a splash but are hits nonetheless. Related Article\nPhoto\nKate Beckinsale in "Love & Friendship." Credit Roadside Attractions\nWithout much fanfare, a handful of small-budget films have been steadily drawing audiences, some for a few months now. The box office numbers may not be high – think $3 million, not $300 million – but the numbers have been quietly climbing all summer. Before the onslaught of fall films, you can catch up with these sleepers, either in theaters or on streaming services.\nA preview of the film. Credit Vid

### 2.2 Print all the recognized entities in (entity, label) tuple format

In [62]:
#Get Entities
print('====Entities====')
print('Type, Name')
for key in TRITJsonResponse:
    if ('_typeGroup' in TRITJsonResponse[key]):
        if TRITJsonResponse[key]['_typeGroup'] == 'entities':
            print(TRITJsonResponse[key]['_type'] + ", " + TRITJsonResponse[key]['name'])

====Entities====
Type, Name
Person, Steve Carell
Position, Captain
Company, The New York Times
ProvinceOrState, New Jersey
Person, A.O. Scott
PublishedMedium, The New York Times
Person, Julian Dennison
Person, Whit Stillman
Position, funny writer
Organization, White House
Journalist, MEKADO MURPHY
Position, brilliant architect
Position, director
Position, future president and first lady of the United States
Country, United States
Person, Stephen Holden
Person, Jesse Eisenberg
Organization, Café Society
Person, Kate Beckinsale
Person, Manohla Dargis
Person, Viggo Mortensen
Person, Richard Tanne
Technology, smartphones
Person, Allen
Person, Yorgos Lanthimos
Position, writer and director
Person, Matt Ross
Movie, July
IndustryTerm, streaming services
Person, Logan Lerman
Person, A. O. Scott
Person, Jane Austen
Person, Sam Neill
Person, Taika Waititi
Person, Colin Farrell
Person, James Schamus
Position, ambassador
Company, Rotten Tomatoes
Country, New Zealand
Company, Roadside Attractions
P

In [63]:
#Get RIC code
print('====RIC====')
print('RIC')
for entity in TRITJsonResponse:
    for info in TRITJsonResponse[entity]:
        if (info =='resolutions'):
            for companyinfo in (TRITJsonResponse[entity][info]):
                if 'primaryric' in companyinfo:
                    symbol = companyinfo['primaryric']
                    print(symbol)

====RIC====
RIC
NYT.N
ZD.OQ


In [64]:
#Print Header
print(symbol)
print('====Topics====')
print('Topics, Score')
for key in TRITJsonResponse:
    if ('_typeGroup' in TRITJsonResponse[key]):
        if TRITJsonResponse[key]['_typeGroup'] == 'topics':
            print(TRITJsonResponse[key]['name'] + ", " + str(TRITJsonResponse[key]['score']))

ZD.OQ
====Topics====
Topics, Score
Entertainment_Culture, 1
